In [1]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Annotated
from langchain_openai import AzureChatOpenAI
import operator
from langgraph.graph import END
import os
# from langchain_community.document_loaders import WebBaseLoader
from langgraph.graph import END, StateGraph, START
# from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from typing import List
from langchain_community.vectorstores import FAISS
from typing_extensions import TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:


llm = AzureChatOpenAI(
    azure_deployment="gpt35",
    api_version="2024-02-01",
    temperature=0,
    max_tokens=50,
    timeout=None,
    max_retries=2,
    )

In [3]:
Prompt="""
You are a helpful assistant in providing topics related to the provided query. The query provided is related to NetApp Storage Grid.

-----------------------------------------------------------------------------------------
NetApp StorageGRID is an object storage solution designed for managing large-scale, unstructured data across multiple 
geographic locations. It provides high durability, availability, and scalability, making it suitable for various use cases,
including data archiving, backup, and content repositories.
---------------------------------------------------------------------------------------
The topics headings and subheadings are provided here:

    # StorageGRID appliances
    # Install StorageGRID on Red Hat Enterprise Linux
    # Install StorageGRID on Ubuntu or Debian
    # Install StorageGRID on VMware
    # Upgrade StorageGRID software
    # Apply StorageGRID hotfix

    # Configure and manage a StorageGRID
    # Administer StorageGRID
    # Manage objects with ILM
    # System hardening
    # Configure StorageGRID for FabricPool
    
    # Use Storage GRID Tenant and Clients
    # Use a tenant account
    # Use S3 REST API
    # Use Swift REST API (deprecated)

    # Monitor and Troubleshoot a Storage Grid
    # Monitor StorageGRID system
    # Troubleshoot StorageGRID system
    # Review audit logs

    # Expand a grid
    # Expand a grid: Overview
    # Plan StorageGRID expansion
    # Gather required materials
    # Add storage volumes
    # Add grid nodes or site
    # Configure expanded system
    # Troubleshoot expansion


    # Maintain a StorageGRID System
    # Maintain your grid: Overview
    # Download Recovery Package
    # Decommission nodes or site
    # Rename grid, site, or node
    # Node procedures
    # Network procedures
    # Host and middleware procedures
    
    # Recover or replace nodes
    # Grid node recovery procedures: Overview
    # Warnings and considerations for grid node recovery
    # Gather required materials for grid node recovery
    # Select node recovery procedure
    # Recover from Storage Node failures
    # Recover from Admin Node failures
    # Recover from Gateway Node failures
    # Recover from Archive Node failures
    # Replace Linux node
    # Replace VMware node
    # Replace failed node with services appliance
    # How technical support recovers a site
    # How to enable StorageGRID in your environment 
------------------------------------------------------------------------------------------------
Output Instruction : Provide only the topics as output.


"""

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# LLM
llm = AzureChatOpenAI(
    azure_deployment="gpt35",
    api_version="2024-02-01",
    temperature=0,
    max_tokens=200,
    timeout=None,
    max_retries=2,
    )

# Prompt

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", Prompt),
        (
            "human",
            f"Here is the initial question: \n\n {Prompt} \n Provide the topics as output.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [29]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [30]:
def get_topics(state):
    """
    Get the topics for which the question would talk about.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
#     documents = state["documents"]

    # Re-write question
    topics = question_rewriter.invoke({"question": question})
    return {"generation": topics}

In [31]:
def get_topics(state):
    """
    Get the topics for which the question would talk about.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
#     documents = state["documents"]

    # Re-write question
    topics = question_rewriter.invoke({"question": question})
    return {"generation": topics}

In [32]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)
workflow.add_node("Analyze Topics", get_topics) 



In [33]:
workflow.set_entry_point("Analyze Topics")
workflow.add_edge("Analyze Topics",END)

In [34]:
app = workflow.compile()

In [35]:
# %%capture --no-stderr
# %pip install --quiet pyppeteer
# %pip install --quiet nest_asyncio
inputs = {"question": "What are the scalability options for the proposed solution?"}
result = app.invoke(inputs)
print(result)

---TRANSFORM QUERY---
{'question': 'What are the scalability options for the proposed solution?', 'generation': '\n- StorageGRID appliances\n- Install StorageGRID on Red Hat Enterprise Linux\n- Install StorageGRID on Ubuntu or Debian\n- Install StorageGRID on VMware\n- Upgrade StorageGRID software\n- Apply StorageGRID hotfix\n- Configure and manage a'}


In [1]:
# !pip install sentence-transformers

storagegrid_dict = {
    "Install, Upgrade and Hotfix StorageGRID": [
        "Installation Overview: Describes the steps required to install the StorageGRID system, including configuring nodes and downloading the recovery package.",
        "Post-Installation Guidelines: Provides instructions for tasks such as DHCP addressing and network configuration changes after completing grid node deployment and configuration.",
        "Virtual Machine Resource Reservation: Details the process of adjusting resources for virtual machines to ensure sufficient RAM and CPU for each grid node.",
        "Temporary Installation Password: Explains how to set and manage a temporary installation password during the deployment of a VMware node.",
        "Upgrading StorageGRID Software: Offers an overview and step-by-step instructions for upgrading the StorageGRID system to a new release.",
        "StorageGRID Installation API: Introduces the API used for performing installation tasks, including sections like config, grid, nodes, provision, recovery, recovery-package, schemas, and sites.",
        "Configuring and Managing Nodes: Discusses creating and placing node configuration files for Ubuntu or Debian deployments, including naming conventions and the content of configuration files.",
        "Troubleshooting Installation Issues: Provides guidance on accessing installation log files and troubleshooting installation problems.",
        "Integration and Configuration Tasks: Lists required and optional tasks after installation, such as creating tenant accounts, configuring groups and user accounts, integrating S3 or Swift API client applications, and configuring ILM rules and policies.",
        "Documentation and Support: Mentions the documentation site for appliances and support for FIPS 140-2 validated cryptography."
    ],
    "Configure and Manage a StorageGRID": [
        "Grid Management API: Describes the use of the Swagger API platform to perform real-time operations in StorageGRID.",
        "Monitoring Data Migration: Provides guidelines for monitoring attributes and managing data migration in the StorageGRID system.",
        "Information Lifecycle Management (ILM): Covers ILM rules and policies for managing object data throughout its lifecycle in StorageGRID.",
        "Admin Group Permissions: Explains the creation and management of admin user groups, including assigning permissions and managing access modes.",
        "Log Files and System Status: Details various log files and system status files, including their contents and how to interpret them."
    ],
    "Use StorageGRID Tenants and Clients": [
        "Tenant Account Overview: Describes the purpose and capabilities of a tenant account in StorageGRID.",
        "Creating a Tenant Account: Steps and requirements for a StorageGRID grid administrator to create a tenant account.",
        "Configuring S3 Tenants: Tasks that can be performed using the Tenant Manager for S3 tenant accounts.",
        "Configuring Swift Tenants: Tasks that can be performed using the Tenant Manager for Swift tenant accounts.",
        "Signing In and Out: Procedures for signing in and out of the Tenant Manager, including both SSO and non-SSO methods.",
        "Tenant Manager Dashboard: Overview of the dashboard's features and the information it displays about tenant accounts.",
        "Storage and Quota Usage: Details on how storage usage and quotas are monitored and managed."
    ],
    "Monitor and Troubleshoot a StorageGRID": [
        "Monitor StorageGRID System: Regular monitoring of the StorageGRID system ensures optimal performance and early detection of issues.",
        "View and Manage the Dashboard: The dashboard provides an overview of system activities and allows for customization to fit specific monitoring needs.",
        "View the Nodes Page: The Nodes page offers detailed metrics for the entire grid, each site, and individual nodes, helping in deeper analysis of system performance.",
        "View Log Files: Log files can be accessed and reviewed to diagnose and troubleshoot issues within the StorageGRID system.",
        "Manage Alerts: Alerts help in detecting, evaluating, and resolving issues with customizable notifications and rules for better system management.",
        "Use SNMP for Monitoring: SNMP can be configured to monitor the StorageGRID system and send notifications based on specific events or thresholds.",
        "Configure Audit Messages and Log Destinations: Audit messages and log destinations can be set up to track system activities and store logs for compliance and review.",
        "Use an External Syslog Server: An external syslog server can be used to collect audit information for centralized logging and monitoring.",
        "Tenant Activity: Monitoring tenant activity helps in understanding resource usage and managing tenant-specific performance issues.",
        "Networking and System Resources: Keeping an eye on networking and system resources ensures that the infrastructure supports the StorageGRID operations effectively."
    ],
    "Expand a StorageGRID": [
        "Plan StorageGRID Expansion: Strategies and considerations for planning an effective expansion of the StorageGRID system.",
        "Guidelines for Adding Storage Volumes: Detailed instructions and limitations on how to add storage volumes to different types of Storage Nodes.",
        "Guidelines for Adding Storage Nodes: Best practices and limitations for adding new Storage Nodes to existing sites.",
        "Guidelines for ADC Service on Storage Nodes: Recommendations for configuring the Administrative Domain Controller (ADC) service on new Storage Nodes.",
        "Add Storage Capacity for Replicated Objects: Considerations for expanding storage to support replicated object data according to the ILM policy.",
        "Add Storage Capacity for Erasure-Coded Objects: Planning and guidelines for adding storage to support erasure-coded object data.",
        "Considerations for Rebalancing Erasure-Coded Data: Insights into the EC rebalance procedure necessary when adding new Storage Nodes to handle erasure-coded data.",
        "Add Metadata Capacity: Instructions on expanding metadata capacity by adding new Storage Nodes to each site."
    ],
    "Maintain a StorageGRID": [
        "Grid Maintenance Overview: Covers tasks like decommissioning nodes, renaming the grid, and maintaining networks.",
        "Maintenance Procedures for Appliances: Provides guidelines for maintaining StorageGRID appliances according to specific hardware instructions.",
        "Download Recovery Package: Instructions on downloading a Recovery Package to restore the system in case of a failure.",
        "Decommission Nodes or Site: Steps to permanently remove grid nodes or an entire site from the StorageGRID system.",
        "Grid Node Decommission Overview: Detailed process for removing one or more grid nodes, except the primary Admin Node.",
        "Considerations for Decommissioning Storage Nodes: Important factors to consider before decommissioning Storage Nodes to ensure proper system functioning.",
        "Connected and Disconnected Site Decommission: Differentiates between removing a connected operational site and a disconnected failed site.",
        "Add or Update Display Names: Procedures for renaming the grid, sites, or nodes without affecting StorageGRID operations.",
        "Monitoring Erasure-Coded Data Repairs: Steps to monitor and retry repairs for erasure-coded data to ensure data integrity.",
        "Decommissioning Connected Grid Nodes: Guidelines for safely decommissioning nodes that are still connected to the grid."
    ],
    "Recovery or Replace Nodes": [
        "Repairing Node Failures: Instructions for repairing failed nodes and tracking repair status using specific commands and tools.",
        "Checking Storage State After Recovery: Steps to verify that the desired state of a recovered storage node is set to online and remains online after a server restart.",
        "Recovering from Storage Volume Failures: Procedures for recovering software-based storage nodes with failed volumes while the system drive remains intact.",
        "Warnings for Storage Volume Recovery: Important warnings and considerations to review before beginning the recovery of failed storage volumes.",
        "Rebuilding the Cassandra Database: Steps to rebuild the Cassandra database as part of the volume recovery process.",
        "Using Recovery Scripts: Details on running specific recovery scripts like sn-remount-volumes and sn-recovery-postinstall.sh to remount, reformat storage volumes, and start services on the storage node.",
        "Restoring Object Data: Guidelines for restoring object data using Grid Manager and monitoring the restoration process.",
        "Monitoring Repair Jobs: Methods to monitor the status of repair jobs for both replicated and erasure-coded data, including using specific commands and tools within the Grid Manager.",
        "Erasure-Coded Data Repair: Specific instructions to monitor and retry failed erasure-coded data repair requests.",
        "Restoration Progress and History: Viewing the progress and history of volume restorations within Grid Manager, including handling failed restoration attempts."
    ]
}


## Sentence Similarity Detector

In [ ]:
import json
import requests

headers = {"Authorization": f"Bearer {api_token}"}

storagegrid_dict.values()
sentences=[]
for i in storagegrid_dict.values():
    sentences=sentences+i

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_key_from_value(d, value):
    for key, val in d.items():
        if val == value:
            return key
    return None 

def topic_finder(topics):

    data = query(
        {
            "inputs": {
                "source_sentence": "Does the solution have automatic failure notification via email, text, and other methods?",
                "sentences":topics
            }
        })


    data_Score=[]
  

    for i in range(0,len(data)):
        data_Score.append((data[i],topics[i]))

    # Sort the list of tuples by the first element in descending order
    data_ = sorted(data_Score, key=lambda x: x[0], reverse=True)

    # Extract the top 3 values and sentences
    top_10 = data_[:80]
    list_subtopics=[top_10[i][1] for i in range(0,3)]
    
    topic_list=[]
    for i in storagegrid_dict.values():
        for j in list_subtopics:
            if j in i:
                topic_list.append(get_key_from_value(storagegrid_dict,i))
    
    return list(set(topic_list))
                
  

In [ ]:
import json
import requests

headers = {"Authorization": f"Bearer {api_token}"}

storagegrid_dict.values()
sentences=[]
for i in storagegrid_dict.values():
    sentences=sentences+i


In [6]:
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_key_from_value(d, value):
    for key, val in d.items():
        if val == value:
            return key
    return None 

In [13]:
def topic_finder(questions,topics):

    data = query(
        {
            "inputs": {
                "source_sentence":questions,
                "sentences":topics
            }
        })
    data_Score=[]
    print(len(topics))
    print(len(data))
    for i in range(0,len(data)):
        data_Score.append((data[i],topics[i]))

    # Sort the list of tuples by the first element in descending order
    data_ = sorted(data_Score, key=lambda x: x[0], reverse=True)

    # Extract the top 3 values and sentences
    top_10 = data_[:80]
    list_subtopics=[top_10[i][1] for i in range(0,5)]
    
    topic_list=[]
    for i in storagegrid_dict.values():
        for j in list_subtopics:
            if j in i:
                topic_list.append(get_key_from_value(storagegrid_dict,i))
    
    return list(set(topic_list))

In [14]:
topic_finder("Does the solution have automatic failure notification via email, text, and other methods?",sentences)

60
89
60


['Maintain a StorageGRID',
 'Monitor and Troubleshoot a StorageGRID',
 'Recovery or Replace Nodes',
 'Install, Upgrade and Hotfix StorageGRID']

In [13]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings

In [ ]:
import os


embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddingada",
    openai_api_version="2024-02-01",
)

In [21]:
loader = PyMuPDFLoader(r"D:\Hackthon\NetApp StorageGrid\Data\8.Recovery or Replace Nodes.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)
db.save_local(r"D:\Hackthon\NetApp StorageGrid\Vector_data\Recovery or Replace Nodes.pdf")


In [79]:


# !pip install langchain_core
# !pip install pygraphviz

In [5]:
data_Score

NameError: name 'data_Score' is not defined

In [91]:
# Pair each element with its index


In [92]:
list_subtopics

['Storage and Quota Usage: Details on how storage usage and quotas are monitored and managed.',
 'Tenant Account Overview: Describes the purpose and capabilities of a tenant account in StorageGRID.',
 'Configuring S3 Tenants: Tasks that can be performed using the Tenant Manager for S3 tenant accounts.',
 'Configuring Swift Tenants: Tasks that can be performed using the Tenant Manager for Swift tenant accounts.',
 'Tenant Activity: Monitoring tenant activity helps in understanding resource usage and managing tenant-specific performance issues.',
 'Creating a Tenant Account: Steps and requirements for a StorageGRID grid administrator to create a tenant account.',
 'Guidelines for Adding Storage Nodes: Best practices and limitations for adding new Storage Nodes to existing sites.',
 "Tenant Manager Dashboard: Overview of the dashboard's features and the information it displays about tenant accounts.",
 'Add Metadata Capacity: Instructions on expanding metadata capacity by adding new Stora

In [19]:
# Given dictionary
storagegrid_dict
# List of numbers to check
sorted_topics 

# Initialize an empty list to store the keys
main_topics = []

# Iterate through the dictionary
for key, value_list in storagegrid_dict.items():
    # Check if any number from the sorted_topics list is in the current list
    if any(number in value_list for number in sorted_topics):
        # Add the key to the new list
        main_topics.append(key)

print(main_topics)


['Install, Upgrade and Hotfix StorageGRID', 'Monitor and Troubleshoot a StorageGRID', 'Maintain a StorageGRID']
